<a href="https://colab.research.google.com/github/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation/blob/main/code_template_explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task

We were given InkML files, which contain metadata and a list of strokes: [(0,0),(1;0)]...  
These are on-line handwritten mathematical expressions, we'll try to recognize them via LG(Labelled Graph) as output.  
This is the sequence of actions performed:  
1) Determine possible stroke combinations.  
2) Remove impossible combinations with a classifier.  
3) Convert each combination to a symbol.  

We'll handle spatial relations later  
We'll also handle the final decision later, but we can add a grammar or language model at that point, 

# Environment setup

## Getting project files

We will be importing the project files from our github repository.

In [1]:
!git clone https://github.com/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation.git

Cloning into 'Master_2_MLVC_Recognize_Handwritten_Equation'...
remote: Enumerating objects: 119843, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 119843 (delta 71), reused 95 (delta 39), pack-reused 119711
Receiving objects: 100% (119843/119843), 46.53 MiB | 17.70 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Checking out files: 100% (209489/209489), done.


In [2]:
%cd Master_2_MLVC_Recognize_Handwritten_Equation/code

/content/Master_2_MLVC_Recognize_Handwritten_Equation/code


We will first show what each script does, and how to interpret their output, then we will show how to use the evaluation scripts, and finally detail how each script functions.

## Data to train our classifiers

[This](https://uncloud.univ-nantes.fr/index.php/s/OdAtErZgxKGjsNy) contains a bunch of symbols (in datasymbol_iso/) and expressions (in FullExpressions/) to help train our future classifiers.



## Showcasing parts of code

### segmenter.py explanation

The first action is to collect the possible stroke combinations, for now we'll simply take every consecutive stroke combinations.  
So if we have 4 strokes in one inkml file, 13 combinations can be done.  

Each line of the output corresponds to a hypothesis: indicating the symbol type, the symbol with index (starting with 1, but dummy value for now since we don't know what symbol the combination could be), then the symbol without index, then the supposed confidence of the model. We also have the strokes used next to these informations.

At this point, this can be optimized by already removed hypotheses take don't make sense, for instance trying to make a symbol with every single stroke is highly irregular and shouldn't happen, so we could remove that combination before calling the other scripts.

The ground truth for the segmentation is available in the original lg files, since the strokes used are listed next to each symbol.

In [5]:
!python3 segmenter.py -i ../data/formulaire001-equation001.inkml -o ../data/example.lg

In [6]:
!python3 segmenter.py -i ../data/formulaire001-equation001.inkml

O,hyp0,*,1.0,0
O,hyp1,*,1.0,1
O,hyp2,*,1.0,2
O,hyp3,*,1.0,3
O,hyp4,*,1.0,4
O,hyp5,*,1.0,0,1
O,hyp6,*,1.0,1,2
O,hyp7,*,1.0,2,3
O,hyp8,*,1.0,3,4
O,hyp9,*,1.0,0,1,2
O,hyp10,*,1.0,1,2,3
O,hyp11,*,1.0,2,3,4
O,hyp12,*,1.0,0,1,2,3
O,hyp13,*,1.0,1,2,3,4



As we've said earlier, we simply keep every consecutive stroke combination as our possible hypotheses.

### Git stuff

In [67]:
#!git stash
#!git stash drop

No local changes to save
No stash entries found.


In [68]:
#!git reset --soft HEAD^ 

In [63]:
#!git pull

remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 3), reused 3 (delta 1), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation
   357bb3271..924dee090  main       -> origin/main
Updating dc38065c7..924dee090
Fast-forward
 code_template_explanation.ipynb | 396 ++++++++++++++++++++++++----------------
 scores_segmentSelector.csv      |   2 +
 2 files changed, 236 insertions(+), 162 deletions(-)
 create mode 100644 scores_segmentSelector.csv


In [40]:
#!git config --global user.email "XXX@gmail.com"
#!git config --global user.name "XXX"

In [69]:
!git add .

In [70]:
!git commit -m "Speed up inference for segmentReco"

[main 3349bd494] Speed up inference for segmentReco
 7 files changed, 24 insertions(+), 272 deletions(-)
 delete mode 100644 code/CROHME-train-iso.py
 create mode 100644 code/graph_train_segmentReco.png
 rewrite code/output.png (99%)
 rewrite code/resultMLP.png (99%)


In [71]:
!git push https://hiddentoken@github.com/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation.git

Counting objects: 9, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 63.01 KiB | 21.00 MiB/s, done.
Total 9 (delta 5), reused 0 (delta 0)
remote: Resolving deltas: 100% (5/5), completed with 5 local objects.
To https://github.com/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation.git
   924dee090..3349bd494  main -> main


### CROHME_train_segmentSelector.py & segmentSelect.py explanation

Since we use neural networks in our overall process as our classifiers/predictors, they need to be trained beforehand. But let's first explain what we're trying to do in this part of the process:  

The script 'segmentSelect.py' takes as input the initial inkml file, alongside the "prototype" lg file: We combine the stroke combinations from the prototype file alongside the inkml stroke data to generate images, then we check whether these images make sense as a symbol, no matter the context.  
This is a classification problem with two possible outputs : Image is valid or invalid.  
We also check the confidence value of the model with a threshold in order to ignore unsure hypotheses, which should boost accuracy somewhat.

Now, for the training part, while we could simply store the weights of the models and import them, we still want to show the specifics of our training due to the characteristics of our data:

While training the model batch per batch, we make sure that each of these batches contain an representative random subset of the original data, since we have a lot less invalid images for training than valid images, while still trying to make sure that the model doesn't excessively consider valid images.  
The rest of the training logic is quite normal, the state of the model is saved whenever we achieve a new validation loss low, and we implemented early stopping to prevent overfitting.

In [51]:
!python3 CROHME_train_segmentSelector.py

cuda:0
('invalid', 'valid')
nb classes 2 , training size 12000, val size 4000, test size 4000
valid invalid valid invalid
AlexNet(
  (layer1): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4), bias=False)
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(96, 384, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
    (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer3): Sequential(
    (0): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (fc): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU()
  )
  (fc1): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(

### segmentSelect.py example

In [61]:
!python3 segmentSelect.py -o ../data/example2.lg ../data/formulaire001-equation001.inkml ../data/example.lg 

In [53]:
!python3 segmentSelect.py ../data/formulaire001-equation001.inkml ../data/example.lg 

O,hyp7,*,0.509894073009491,2,3
O,hyp8,*,0.9908795356750488,3,4
O,hyp10,*,0.7905758023262024,1,2,3
O,hyp12,*,0.7709046602249146,0,1,2,3
O,hyp13,*,0.6058663725852966,1,2,3,4



This script takes as input the initial inkml file, alongside the "prototype" lg file: We combine the stroke combinations alongside the inkml data to generate images, then we check whether these make sense as a symbol, no matter the context.

For now this simply gives a fully random probability that the images are valid symbols.

Need to change two things in there, first adding a classifier to check the validity of supposed symbols,  
then change the threshold based on empirical evidence or what I feel like at the time.

### segmentReco.py explanation

In [55]:
!python3 CROHME_train_segmentReco.py

cuda:0
['theta', 'n', 'w', 'z', 'S_', 'x', 'h', ')', 'dot', 'beta', 'gamma', 's', 'd', '+', 'm', '1', 'V_', '9', 'sqrt', '}', 'rightarrow', 'r', 'b', 'F_', 'u', 't', 'leq', 'tan', 'sin', 'alpha', 'exists', 'div', 'A_', 'pipe', '(', 'H_', 'i', 'o', '3', 'forall', 'N_', 'times', 'M_', 'E_', 'Delta', 'lambda', 'L_', 'neq', 'c', ',', 'a', '5', 'q', 'R_', '0', 'Y_', 'sum', 'div_op', '[', 'log', '8', 'C_', 'lim', 'j', '4', 'lt', 'infty', 'pm', '=', 'sigma', 'phi', 'I_', 'gt', 'G_', 'cos', 'prime', '2', 'p', 'B_', 'geq', 'in', 'v', ']', 'k', 'l', '{', 'ldots', 'P_', '6', 'int', 'pi', 'g', 'mu', 'T_', 'e', '!', 'X_', 'y', 'f', '-', '7']
nb classes 101 , training size 6000, val size 2000, test size 2000
theta     w     z theta
AlexNet(
  (layer1): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4), bias=False)
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(96, 384, kernel_si

In [62]:
!python3 symbolReco.py -o ../data/example3.lg ../data/formulaire001-equation001.inkml ../data/example2.lg

In [ ]:
!python3 symbolReco.py ../data/formulaire001-equation001.inkml ../data/example2.lg

### selectBestSeg.py explanation

## Evaluating

use lgeval and show some tables and graphics.

TODOLIST: check that everything works: X  
Apply evaluation script (import relevant things here): X  
Nitpick a bunch of things:  X  
Write some things on notebook: X  
